# <span style="color:gold">**Geopandas**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este tutorial, revisaremos las funcionalidades básicas de <span style="color:gold">Geopandas</span>.

Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.


## **¿Qué es Geopandas?**
***
<span style="color:gold">Geopandas</span> es una librería de Python que extiende las funcionalidades de <span style="color:gold">Pandas</span> para que pueda ser usado en el análisis de datos geoespaciales.\
Las operaciones espaciales (con objetos geométricos) son realizados a través de <span style="color:gold">Shapely</span>.

### **1. `GeoDataFrame` y `GeoSeries`**

La estructura de datos principal en Geopandas es el `GeoDataFrame`, una subclase del `DataFrame` de Pandas, que puede almacenar columnas de geometría y realizar operaciones espaciales.\
Una `GeoSeries` es una subclase de la `Series` de Pandas, que maneja las geometrías.\
En resumen, el `GeoDataFrame` es una combinación de las `Series` de Pandas, con datos tradicionales (numéricos, lógicos, etc.) y las `GeoSeries` de Geopandas, con geometrías (puntos, líneas, polígonos, etc.).

<img src="https://geopandas.org/en/stable/_images/dataframe.svg" style="background:white" width="750px"/>

Una `GeoSeries` puede contener cualquier tipo de geometría y tiene un atributo `crs`, que corresponde al sistema de coordenadas usado para representar y proyectar la información espacial (CRS es una abreviación de Coordinate Reference System).\
En resumen, cada `GeoSeries` en un `GeoDataFrame` puede estar en una proyección diferente, permitiendo tener múltiples proyecciones de la misma geometría.

Solo se puede tener una `GeoSeries` como geometría activa dentro del `GeoDataFrame`, esto significa que todas las operaciones geométricas realizadas en el `GeoDataFrame` operan sobre esta columna activa.

### **2. ¿Cómo abrir un shapefile en Python?**

Podemos abrir un shapefile usando Geopandas, a través de la función `read_file`:
> Abriremos el contenido de `countries` dentro de la carpeta `files`, y lo asignaremos a la variable `gdf`.

In [ ]:
# Desactivamos las advertencias de Python, innecesarias en este tutorial
import warnings
warnings.filterwarnings("ignore")

import geopandas as gpd

In [ ]:
gdf = gpd.read_file("files/countries")

Usaremos el método `head` (igual que en Pandas) para observar las 5 primeras filas del GeoDataFrame:

In [ ]:
gdf.head()

In [ ]:
print(f"Total de países: {len(gdf)}")

El dataset de `countries` contiene información general acerca de 177 países: población, continente al que pertenecen, nombre, abreviatura, PBI y una geometría de polígono con la forma de cada país.

### **3. ¿Cómo visualizar un shapefile?**

Para visualizar los objetos geométricos que representan el área de cada país, usaremos el método `plot`:

In [ ]:
gdf.plot();

Podemos modificar los detalles del gráfico con los argumentos `figsize` y `color`:

In [ ]:
gdf.plot(figsize=(10, 10), color="green");

Podemos agregar mayor flexibilidad si lo combinamos con Matplotlib:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(10, 10))

# Polígonos de los países
gdf.plot(ax=ax, color="green")

# Límites de la figura
ax.set_xlim(-90, -30)
ax.set_ylim(-60, 15)

# Grilla
ax.grid(lw=0.5, color="black", alpha=0.5)

plt.show()

### **4. Crear y guardar un shapefile con Python**

Empezaremos seleccionando las filas que correspondan al continente de Sudamérica `South America`:

In [ ]:
south_america = gdf[gdf["continent"] == "South America"]

Para guardar información vectorial con Geopandas, podemos usar la función `to_file`:
> Para guardarlo en un shapefile, debemos colocar la extensión .shp, por ejemplo: `south_america.shp`.\
> Si por otro lado solo colocamos el nombre, sin la extensión, se creará una carpeta que contendrá dicho shapefile.

In [ ]:
south_america.to_file("files/south_america")

Volvemos a cargar el shapefile:

In [ ]:
gpd.read_file("files/south_america")

### **5. Funciones adicionales en Geopandas**
Podemos obtener el sistema de coordenadas del GeoDataFrame usando el atributo `crs`:

In [ ]:
gdf.crs

Es importante tener en cuenta el sistema de coordenadas de cada objeto geométrico que utilicemos para evitar errores de proyección.

El método `explore` muestra un mapa interactivo del GeoDataFrame:
> El parámetro `column` selecciona la columna que queremos visualizar.\
> El parámetro `legend` es usado para agregar una leyenda al mapa.

In [ ]:
gdf.explore(column="pop_est", legend=False)

Podemos generar las columnas de área, perímetro y centroide de cada polígono usando los atributos `area`, `length` y `centroid` respectivamente:

In [ ]:
gdf["area"] = gdf.area
gdf["perimeter"] = gdf.length
gdf["centroid"] = gdf.centroid

In [ ]:
gdf.head()

Por último, graficaremos los polígonos y sus centroides dentro de una sola figura:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))

gdf["geometry"].plot(ax=ax, color="lightgreen", edgecolor="black")
gdf["centroid"].plot(ax=ax, color="red", marker=".", markersize=10)

plt.show()

### **6. ¿Cómo crear un GeoDataFrame a partir de un DataFrame con coordenadas?**

Cargaremos el CSV de sismos llamado `sismos.xlsx`:

In [ ]:
import pandas as pd

In [ ]:
sismos = pd.read_excel("files/sismos.xlsx")

In [ ]:
sismos.head()

Podemos usar las columnas de longitud y latitud para crear geometrías de puntos usando la función `points_from_xy`.\
Guardaremos este resultado en una nueva columna llamada `geometry`:

In [ ]:
sismos["geometry"] = gpd.points_from_xy(sismos["longitud (º)"], sismos["latitud (º)"])

In [ ]:
sismos.head()

Ahora, transformaremos el DataFrame en un GeoDataFrame usando la función `GeoDataFrame`:
> Usaremos el sistemas de coordenadas geográfico WGS84 bajo la denominación EPSG:4326.

In [ ]:
sismos = gpd.GeoDataFrame(sismos, geometry=sismos.geometry, crs="EPSG:4326")

In [ ]:
sismos.head()

Por último, graficaremos la ubicación de todos los sismos de magnitud mayor a 5 registrados en el Perú:
> Usaremos los polígonos de cada país en `gdf` para complementar el gráfico.

In [ ]:
sismos_filtrado = sismos[sismos["magnitud (M)"] >= 5.0].copy()

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(8, 10))

# Polígonos de los países
gdf.plot(ax=ax, color="lightgreen", edgecolor="black")
sismos_filtrado.plot(ax=ax, markersize=2, color="red")

# Límites de la figura (centrado en Perú)
ax.set_xlim(-83, -68)
ax.set_ylim(-19, 1)

# Grilla
ax.grid(lw=0.5, color="black", alpha=0.5)

# Título
ax.set_title("Sismos registrados en Perú (Mw$\geq$5.0)", fontsize=16)

plt.show()

***